# Hands-On Workbook: Deep Learning in Medicine

Lecturer: Susanne Suter (susanne.suter@fhnw.ch)

Reference: this notebook was inspired by https://appliedmldays.org/workshops/machine-learning-for-smart-dummies

### General information to run this notebook

- For this notebook, the Tensorflow image is required on the FHNW JupyterHub server (https://jhub.cs.technik.fhnw.ch/).
- Locally, the dependencies can be loaded via the Docker Tensorflow image (https://hub.docker.com/r/tensorflow/tensorflow/).
- The notebook can alternatively be used on Google Colab (https://research.google.com/colaboratory/).
- GPU resources are an advantage (e.g. via Google Colab), but are not required.
- Video tutorial for this notebook: https://tube.switch.ch/videos/p456sXZPq6 (Attention: there is a small bug with the sklearn Confusion Matrix).

# Tumor Classification using a CNN

In this exercise, we will create and tune a convolutional neural network (CNN) that is able to detect the tumors in brain images acquired using MRI. 

You will be gently guided through every step to create the trained CNN classifier. Parameters that can be tuned are marked with <font color='blue'>TODO</font>. Questions to be answered are marked with <font color='blue'>Question</font>.

The ground truth data is not perfect, hence, this will allow you to capture the essential points about the ground truth data generation.

The ground truth data contains of the labels (answers) inserted into the CNN that are used for training. That is, the ground truth represents the true y values.

<font color='blue'>Question</font> Is this a regression or classification task that we model?

In [2]:
import matplotlib.pyplot as plt
%matplotlib inline
from skimage import io
from skimage.transform import rescale, resize, downscale_local_mean
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

import fnmatch
import numpy as np
import os, glob

import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.constraints import MaxNorm
from tensorflow.keras.optimizers import SGD, Adam
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

## Importing Data

This time, we will import the data for this exercise directly from a git repository. 

The MRI images (with and without tumor) are for teaching purposes prepared from the below source data. They are already sorted in two directories for our two classes to classify/predict.

The ownership of the data is: "Brain MRI Images for Brain Tumor Detection"
https://www.kaggle.com/navoneel/brain-mri-images-for-brain-tumor-detection
Navoneel Chakrabarty (2019), version 1.0. 

In [3]:
# Import MRI data (includes independent benchmark data)
# Note: only import this data once (not every time)

!rm -fr MRI_data* # removes the directory MRI_data in case it already exists

!git clone https://github.com/susuter/MRI_data.git # clones the git repository

'rm' is not recognized as an internal or external command,
operable program or batch file.
fatal: Too many arguments.

usage: git clone [<options>] [--] <repo> [<dir>]

    -v, --verbose         be more verbose
    -q, --quiet           be more quiet
    --progress            force progress reporting
    -n, --no-checkout     don't create a checkout
    --bare                create a bare repository
    --mirror              create a mirror repository (implies bare)
    -l, --local           to clone from a local repository
    --no-hardlinks        don't use local hardlinks, always copy
    -s, --shared          setup as shared repository
    --recurse-submodules[=<pathspec>]
                          initialize submodules in the clone
    --recursive ...       alias of --recurse-submodules
    -j, --jobs <n>        number of submodules cloned in parallel
    --template <template-directory>
                          directory from which templates will be used
    --reference <repo>   

The loaded data contains three folders with MRI brain images in png format. You can explore them by browsing the directories next to this notebook file.

* Folder `yes` contains images with tumors
* Folder `no` contains images without tumors
* Folder `benchmark` contains independent test images

Note: Ideally, the input images are all of the same pixel width and height. This way, we profit from the best resolution available from this data.

<font color='blue'>Question</font>: What are potential problems, when the input data is not of the same image size?
 


In [4]:
# Show an image of each dataset to make sure that the data is correctly loaded 
# This gives an idea on how the images look

# TODO: display other images yourself
img_1 = io.imread("MRI_data/pos/Pos1.png")
img_2 = io.imread("MRI_data/neg/Neg2.png")

fig = plt.figure(figsize=(10,10))
ax1 = fig.add_subplot(1,2,1)
ax1.imshow(img_1)
ax1.set_title("Example image with tumor")
ax2 = fig.add_subplot(1,2,2)
ax2.imshow(img_2)
ax2.set_title("Example image without tumor")


# Display the size of the images. In python this is called "shape"
# (pixel rows, pixel columns)

print( "Shape of image with tumor: ")
print( img_1.shape )

print( "Shape of image without tumor: ")
print( img_2.shape )

FileNotFoundError: No such file: 'Z:\brain-health\fhnw_ds_fs22_cdl1_cml1_Brain_Health_Challenge\notebooks\0_eda\MRI_data\pos\Pos1.png'

<font color='blue'>Question</font>: Do you see the tumor in the image?

In [ ]:
# Helper function to count number of png images in given directory

def count_pngs_in_directory(dir_name):
  print( "Number of png images in directory '", dir_name, "': ")
  !cd $dir_name; ls -l *.png | wc -l


In [ ]:
# Print the number of source images available for each dataset

count_pngs_in_directory("MRI_data/pos")
count_pngs_in_directory("MRI_data/neg")


## Augmenting the Available Data

Data augmentation is performed in order to make your ground truth dataset more diverse such that the computed algorithm is more robust to new data - not used for the training. Typical data augmentation modifications include adjusting brightness, color, contrast, distorions, adding noise, zooming, rotations, or mirroring.

There are various ready made data augmentor tools available. In case you need to augment also your labels, make sure you choose an augmentor tool that is able to do so. The one presented here can only augment raw images.  

In [ ]:
# Import Augmentor (library to create more distorted images)
!pip install Augmentor
import Augmentor

# Define a function to create augmented images images
# TODO: change parameters of augmentor
def build_augmented_images(path_folder, n_samples):
    p = Augmentor.Pipeline(path_folder)
    p.rotate(probability=0.3, max_left_rotation=4, max_right_rotation=4)
    p.zoom(probability=0.3, min_factor=0.7, max_factor=1.2) 
    p.random_distortion(probability=0.3, grid_width=4, grid_height=4, magnitude=6)
    p.random_brightness(probability=0.8, min_factor=0.5, max_factor=2)
    p.random_color(probability=0.3, min_factor=0.5, max_factor=1.5)
    p.random_contrast(probability=0.3, min_factor=0.8, max_factor=1.2)

    p.sample(n_samples, multi_threaded=False)

In [ ]:
# Remove previous augmented images, if any (in case you run it twice) 
!rm -fr MRI_data/pos/output 
!rm -fr MRI_data/neg/output

# Number of augmented images

n_y = 500 # number of augmented tumor images
n_n = 500 # number of augmented non-tumor images

# TODO: choose your own number of augmented images for the two classes

# Hint 1: if you start with smaller numbers, the CNN will be trained faster
# However, the more images you add, the better accuracy/performace you will achieve
# The performance of the CNN can be evaluated in the section "Prediction Performance"
# Hint 2: consider whether you want to select the same or an unequal number of 
# images for the two classes

# Create augmented images
build_augmented_images("MRI_data/pos", n_y)
build_augmented_images("MRI_data/neg", n_n)
    

In [ ]:
# Check number of augmented images of each category

count_pngs_in_directory("MRI_data/pos/output")
count_pngs_in_directory("MRI_data/neg/output")

In [ ]:
# Optionally: list selected augmented images (for checking)

!ls -l MRI_data/pos/output/*Pos10*


In [ ]:
# Function to display example augmented images

def show_augmented_images_example(image_type, image_number):
  
  base_path = os.path.join('MRI_data', image_type.lower())
  augmentation_path = os.path.join(base_path, 'output')

  original_filename = os.path.join(base_path, image_type + image_number + '.png')
  if not(os.path.exists(original_filename)):
    print("select a valid image number or type")
    return

  original = io.imread(original_filename)
  pattern = '*' + image_type + image_number + '*.png';
  augmented = fnmatch.filter(os.listdir(augmentation_path), pattern)

  fig = plt.figure(figsize=(20,10))
  ax1 = fig.add_subplot(1,4,1)
  ax1.set_title("Original")
  ax1.imshow(original)

  if len(augmented) > 0:
    augmented1 = io.imread(os.path.join(augmentation_path, augmented[0]))
    ax2 = fig.add_subplot(1,4,2)
    ax2.imshow(augmented1)
    ax2.set_title("Augmented")

  if len(augmented) > 1:
    augmented2 = io.imread(os.path.join(augmentation_path, augmented[1]))
    ax3 = fig.add_subplot(1,4,3)
    ax3.imshow(augmented2)
    ax3.set_title("Augmented")
  
  if len(augmented) > 2:
    augmented3 = io.imread(os.path.join(augmentation_path, augmented[2]))
    ax4 = fig.add_subplot(1,4,4)
    ax4.imshow(augmented3)
    ax4.set_title("Augmented")


In [ ]:
# TODO: display exemplar augmented images: change image number and image type
image_number = '85'
image_type = 'Neg' # 'Pos' or 'Neg'
show_augmented_images_example(image_type, image_number)

## Data Preparation

In this part, we want to make sure that the data is correctly formatted.
For example, we need each image to be of the same size with respect to the images dimensions (WIDTH and HEIGHT) and the number of color channels (DEPTH). At the same time, we add the prepared image to a list with our desired file format.

In [ ]:
# Generate the image lists for both classes, tumor and non-tumor
image_list_y = glob.glob('MRI_data/pos/output/*.png')
image_list_n = glob.glob('MRI_data/neg/output/*.png')

In [ ]:
# Define the function that pre-processes the images for the training

# This function does the resize and adds the image to the training set
def img_preprocessing(image_list, label, X_, y_):

    i = 0
    for image in image_list:
    
        if i%100 == 0: print("pre-processing image ",i," ...")
    
        img = io.imread(image)

        img = np.array(img)

        # TODO: Optionally change the pixel WIDTH and HEIGHT

        # Resize the image; make sure all images have same size
        pr = 250 # pixel rows (HEIGHT)
        pc = 250 # pixel columns (WIDTH)
        img = resize(img, (pr, pc), anti_aliasing=True)

        if i == 0:
            WIDTH, HEIGHT = np.array(img).shape
            print("image size:",WIDTH,HEIGHT)

        # Show the first 5 images
        if i < 5:
            plt.imshow(img)
            plt.show()
    
        # Append the pre-processed images to the training set
        X_.append(img)
        y_.append(label)
    
        i = i+1

In [ ]:
# Initialize the training set
X_v0 = [] # images
y_v0 = [] # labels (ground truth)

# Pre-process images and add them to training set
print("pre-processing tumor images and add them to training set...")
img_preprocessing(image_list_y, [1,0], X_v0, y_v0)
print("pre-processing non-tumor images and add them to training set...")
img_preprocessing(image_list_n, [0,1], X_v0, y_v0)

print("Training set is ready.")

In [ ]:
# Change format of the training set from list to numpy array
X = np.array(X_v0) # images
y = np.array(y_v0) # labels

# reshape for tensorflow compatibilty
X = X.reshape(X.shape[0], X.shape[1], X.shape[2], 1)

print(X.shape,y.shape)

<font color='blue'>Question</font>: How do you interpret the output of calling `shape` on X or y? How does the output differ from the above output?

In [ ]:
# Defines a function that shows an image from the image_set
def show_img(image_set, i):
    img = image_set[i, :, :]
    img = img.reshape(img.shape[0], img.shape[1])
    plt.imshow(img)
    plt.show()

In [ ]:
# Show example images from the training set and check their labels
# Label -> what we are predicting
# Label [1 0] -> tumor
# Label [0 1] -> non-tumor

def show_label(i):
  show_img(X,i)
  print("Label: ",  y[i,:])

# TODO: select other image indices below

# Hint 1: the index range depends on your chosen number of selected augmented images
# Hint 2: the tumor images have the lower indices since they were added to the list first

# tumor images
show_label(10) 
show_label(22) 

# non-tumor images
show_label(510) 
show_label(563) 


In [ ]:
# Print X (images) and y (labels) shapes
print(X.shape,y.shape)

In [ ]:
# Print X (images) and y (labels) values
# print("Images X: ",X[:5])
print("Labels y: ",y[:5])

## Split Data into Training Set and Validation Set

During the training, the CNN needs certain cross-validation data to compute the loss. What in return makes it possible to update the parameters (weights) of the trained CNN in order to make it perform better. 

Therefore, we split our ground truth data into a training set and a validation set. Typically, we choose about 20% of the ground truth as validation set. 

The terminology of the validation set can vary. E.g., it is also called test set. 

In [ ]:
# Split the data from the preprocessed set into a train and test set

X_train, X_validation, y_train, y_validation = train_test_split(X, y, test_size=0.20)

print('X_train shape: ', X_train.shape)
print('X_validation shape: ', X_validation.shape)
print('y_train shape: ', y_train.shape)
print('y_validation shape: ', y_validation.shape)

print('Train and test datasets are ready!')

## CNN Model Definition

In this section, we define the architecture of our CNN model. That means, we define how many and what type of layers the CNN has. We furthermore define other parameters such as the activation function or our regularization (dropout). 

In [ ]:
# Set random seed for reproducibility
seed = 7
np.random.seed(seed)

def define_model(num_classes,epochs):
    # Create the CNN model
    model = Sequential()

    # TODO: Play with the parameters of the layers
    
    # Hint: Parameters to play with are: 
    # convolutions, activation functions, dropout, max pooling, ...

    # Layer 1 (convolutional plus max pooling)
    model.add(Conv2D(4, (5, 5), input_shape=(X.shape[1], X.shape[2], 1), padding='same', activation='relu', kernel_constraint=MaxNorm(3)))
    model.add(Dropout(0.2))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    # Layer 2 (convolutional plus max pooling)
    model.add(Conv2D(4, (3, 3), activation='relu', padding='same', kernel_constraint=MaxNorm(3)))
    model.add(Dropout(0.2))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    # TODO: Optionally add additional convolutional layers
    # ...

    # Additional dense layers (fully connected)    
    model.add(Flatten())
    model.add(Dense(num_classes, activation='softmax'))
 
    # TODO: Optionally choose other (custome-defined) optimizers
    
    #lrate = 0.005
    #decay = lrate/epochs
    #sgd = SGD(lr=lrate, momentum=0.9, decay=decay, nesterov=False)
    #adam = Adam(lr=lrate, beta_1=0.9, beta_2=0.999, epsilon=None, decay=decay, amsgrad=False)
    adam = Adam()
    
    # Prepares the model and defines the loss and optimizer
    model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])
    
    print(model.summary())
    
    return model


In [ ]:
# Define the duration of the training process, which is given in epochs
# In each epoch the model learns once the whole dataset

# TODO: play with the number of epochs that the model is trained for
epochs = 10

#Create the model
model=define_model(2,epochs)

## CNN Training

In [ ]:
# Train the model
history=model.fit(X_train, y_train, validation_data=(X_validation, y_validation), epochs=epochs, batch_size=32)

# Plot accuracy vs epochs
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()
# Plot cost function vs epochs
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model cost function')
plt.ylabel('cost function')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

# Final evaluation of the model
scores = model.evaluate(X_validation, y_validation, verbose=0)
print("______________________________")
print("Validation set accuracy: %.2f%%" % (scores[1]*100))
print("______________________________")
# Save the model to a json file
model_json = model.to_json()
with open("model_MRI_v1.json", "w") as json_file:
    json_file.write(model_json)
model.save_weights("model_MRI_v1.h5")

## Prediction Performance

In [ ]:
# Predictions on the validation sample
p_validation = (model.predict(X_validation)>0.5).astype('int32')

# Predictions on the training sample
p_train = (model.predict(X_train)>0.5).astype('int32')

print('Confusion matrix for the train set:')
print(confusion_matrix(y_train[:,0], p_train[:,0]))

print('Confusion matrix for the validation set:')
print(confusion_matrix(y_validation[:,0], p_validation[:,0]))

print('[[true positives, false negatives]')
print('[false positives, true negatives]]')


In [ ]:
def show_example_prediction(X_, y_,p_,i):
  show_img(X_,i)
  print("Label:       ",y_[i,:])
  print("Prediction:  ",p_[i,:])

# TODO: choose other image indices below (last parameter)
# TODO: find false positives and false negatives
show_example_prediction(X_validation,y_validation,p_validation,100)
show_example_prediction(X_validation,y_validation,p_validation,102)
show_example_prediction(X_validation,y_validation,p_validation,104)


## Independent Benkmark Test Set

Now, the trained CNN model is evaluated on an independent test dataset, which was not used for to calculated the loss during the training.

To differentiate from the terminology test set, which was used during the training, we use here the terminology benchmark test set.

In [ ]:
# Prepare the benchmark test set

import glob

# Generate the image lists for both benchmark classes, female and male
# The source data was downloaded previously from the repository
image_list_bench_f = glob.glob('MRI_data/benchmark/pos/*.png')
image_list_bench_m = glob.glob('MRI_data/benchmark/neg/*.png')

# Initialize the benchmark set
X_bench_v0 = [] # images
y_bench_v0 = [] # labels (ground truth)

# Pre-process images and add them to benchmark set
print("pre-processing tumor images and add them to benchmark set...")
img_preprocessing(image_list_bench_f, [1,0], X_bench_v0, y_bench_v0)
print("pre-processing non-tumor images and add them to benchmark set...")
img_preprocessing(image_list_bench_m, [0,1], X_bench_v0, y_bench_v0)

# Change format of the benchmark set from list to numpy array
X_bench = np.array(X_bench_v0) # images
y_bench = np.array(y_bench_v0) # labels

print("Benchmark set is ready.")


In [ ]:
# Predict the images of the benchmark test set
X_bench = X_bench.reshape(X_bench.shape[0], X_bench.shape[1],X_bench.shape[2], 1)
p_bench = (model.predict(X_bench)) # probabilities
p_bench_c = (p_bench>0.5).astype('int32') # classification


## Performance of Benchmark Set

In [ ]:
# Define functions to calculate performance

# note: handling division by zero for readability not implemented

# accuracy
def calc_accuracy(tp,tn,n):
  return (tp+tn)/n

# harmonic mean of precision and recall
def calc_f1_score(tp,fp,fn):
  return (2*tp)/(2*tp + fp + fn)


In [ ]:
# Calculate performance

from sklearn.metrics import confusion_matrix

print('Confusion matrix for the benchmark test set')
cf = confusion_matrix(y_bench[:,0], p_bench_c[:,0])
print(cf)
print('[[true negatives, false negatives]')
print('[false positives, true positives]]')

n = y_bench.shape[0]
tp = cf[1,1]
tn = cf[0,0]
fp = cf[1,0]
fn = cf[0,1]

print("Total samples        : ",n)
print("True positives (tp)  : ",tp) # it is really a tumor
print("False negatives (fn) : ",fn) # it is classified as non-tumor, but is a tumor
print("False positives (fp) : ",fp) # it is classified as tumor, but is a non-tumor
print("True negatives (tn)  : ",tn) # it really is not a tumor, i.e., it is a non-tumor

print("Accuracy             : ",calc_accuracy(tp,tn,n))
print("F1 score             : ",calc_f1_score(tp,fp,fn))


In [ ]:
def show_example_prediction(X_, y_,p_,i):
  show_img(X_,i)
  print("Label:       ",y_[i,:])
  print("Prediction:  ",p_[i,:])

# tumor (smaller indices - depends on your total ground truth size)
show_example_prediction(X_bench,y_bench,p_bench_c,0)
show_example_prediction(X_bench,y_bench,p_bench_c,1)
show_example_prediction(X_bench,y_bench,p_bench_c,2)

# non-tumor (larger indices - depends on your total ground truth size)
show_example_prediction(X_bench,y_bench,p_bench_c,5) 
show_example_prediction(X_bench,y_bench,p_bench_c,6) 
show_example_prediction(X_bench,y_bench,p_bench_c,7) 

# Challenge: Optimize the CNN Yourself

* Challenge:
optimize the model to have the highest accuracy on the test set (test set accuracy)

* Parameters to vary:
  * Number of augmented images / number of images in training set
  * Ratio between tumor and non-tumor images
  * Selection of images for training
  * (Resizing of images)
  * Epochs
  * For the convolutional layers:
    * Number of convolutional layers
    * Number of filters in a layer
    * Size of layers (3x3, 5x5, ...)
    * Activation functions
  * Adjust the optimizer and add manual parameters (gradient descent)

* Rule of thumb: number of parameters of the model should be lower than the training set size.

* Hint: Good configurations of the CNN model are typically found by searching the internet for existing working architectures.